In [95]:
!pip install spacy tabula https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)


In [96]:
# Packages
import os
import re
import pdfplumber
import numpy
import pandas as pd
import spacy
from datetime import datetime

In [136]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load the provider info CSV for ending keywords
provider_info = pd.read_csv('provider.csv')

# Load the company info CSV for ticker validation and company metadata
company_info = pd.read_csv('company_info.csv')  # Replace with the actual path

company_info = company_info.drop_duplicates(subset='Ticker Symbol')

# Create a dictionary to map ticker symbols to company name and industry
ticker_map = company_info.set_index('Ticker Symbol')[['Company Name', 'Industry']].to_dict(orient='index')

In [137]:
provider_info

,Provider,file_name,Ending,Price Target,Recommendation
0,BGC Partners,BGC,Disclosures Appendix,Price Target ($) {price},"{RATING} ({TICKER},$"
1,Needham,Needham,Analyst Certification,Price Target ${price},Stock Rating {RATING}
2,BTIG,BTIG,Appendix: Analyst Certification and Other Impo...,12 month target ${price},{RATING} 52 week range
3,Wells Fargo,Wells_Fargo,Required Disclosures,Overweight/${price},{Rating}/$
4,Barclays,Barclays,ANALYST(S) CERTIFICATION(S),Price Target USD {price},Stock Rating {RATING}
5,JP Morgan,JP_Morgan,Analyst Certification,Price Target ({Letters & numbers}): ${price},{Rating} {TICKER}
6,Brean Capital,Brean_Capital,Analyst Certification,PT: $ {price},{Rating} PT:$
7,Hilliard Lyons,Hilliard_Lyons,Analyst Certification,Price Target {price},NSYE -- {Rating} --
8,Alliance Global Partners,Alliance_Global_Partners,Imporant Research Disclosures,Price Target ${price},{Rating} (Ticker:
9,Mizuho Securities,Mizuho_Securities,IMPORTANT DISCLOSURES,Price Target ${price},Rating Buy


In [203]:
df_missing_paragraphs.value_counts("provider")

provider
Barclays                          12
Gilford Securities Inc             8
Pivotal Research Group             6
JP Morgan                          5
Wells Fargo                        4
IFS Securities                     2
Brean Capital                      1
IBI Investment House               1
Spartan Capital Securities LLC     1
Name: count, dtype: int64

In [194]:
# Only relevant to check font type and size for
import pdfplumber

def extract_words_with_formatting(page):

    # Extract words with their bounding boxes
    words = page.extract_words(extra_attrs=["fontname", "size"])

    formatted_words = []
    for word in words:
        formatted_words.append({
            "word": word["text"],
            "font": word.get("fontname", "Unknown"),
            "size": word.get("size", "Unknown"),
            "x0": word["x0"],
            "x1": word["x1"],
            "top": word["top"],
            "bottom": word["bottom"]
        })
    return formatted_words


# Example usage with pdfplumber
pdf_path = "/Users/oskarroeske/Desktop/Analyst_Reports/Production/20150504_Brean_Capital_FCX_FCX-_Maintain_Hold_on_FCX-_Capex_Hike_Disappointing_Cou.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Page {page_number}:")
        formatted_words = extract_words_with_formatting(page)
        for word_info in formatted_words:
            print(
                f"Word: '{word_info['word']}', Font: {word_info['font']}, Size: {word_info['size']}, "
                f"Position: ({word_info['x0']}, {word_info['top']} - {word_info['x1']}, {word_info['bottom']})"
            )


Page 1:
Word: 'REVIEW', Font: EBCBCE+Calibri, Size: 12.0, Position: (42.519000150000004, 38.78200020999998 - 81.21900015, 50.78200020999998)
Word: 'OF', Font: EBCBCE+Calibri, Size: 12.0, Position: (83.93100015000002, 38.78200020999998 - 97.38300015, 50.78200020999998)
Word: 'EARNINGS', Font: EBCBCE+Calibri, Size: 12.0, Position: (100.09500015000002, 38.78200020999998 - 150.83100015000002, 50.78200020999998)
Word: 'Freeport-McMoRan', Font: EBCBCF+Calibri-Bold, Size: 12.0, Position: (36.85, 64.51899910999998 - 135.034, 76.51899910999998)
Word: 'Inc.', Font: EBCBCF+Calibri-Bold, Size: 12.0, Position: (137.746, 64.51899910999998 - 155.59, 76.51899910999998)
Word: 'April', Font: EBCBCE+Calibri, Size: 10.0, Position: (36.85, 78.83299987999999 - 55.940000000000005, 88.83299987999999)
Word: '27,', Font: EBCBCE+Calibri, Size: 10.0, Position: (58.2, 78.83299987999999 - 70.81, 88.83299987999999)
Word: '2015', Font: EBCBCE+Calibri, Size: 10.0, Position: (73.07000000000001, 78.83299987999999 - 93.3

In [195]:
# Test new Structure

patterns = {
    "APP Securities Pty Ltd": {
        "price_patterns": {
            "primary": r"TARGET PRICE (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 8.25},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Alliance Global Partners": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Imporant Research Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]ArialMT(-BoldMT)?", "font_size": 8.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Barclays": {
        "price_patterns": {
            "primary": r"Price Target USD (\d+(\.\d+)?)",
            "secondary": r"Price Target: USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Stock Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"ANALYST\(S\) CERTIFICATION\(S\)",
                            r"Analyst\(s\) Certification\(s\)"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?(Regular,Bold)?", "font_size": 9.0},
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01},
        ]
    },
    "BGC Partners": {
        "price_patterns": {
            "primary": r"Price Target \(\$\) (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) \(\w+,",
        },
        "ending_patterns": [r"Disclosures Appendix"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Brean Capital": {
        "price_patterns": {
            "primary": r"PT: \$ (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) PT:\$",
        },
        "ending_patterns": [r"Analyst Certification",r"Important Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?(-Italic)?", "font_size": 8.00},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?(-Italic)?", "font_size": 9.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "BTIG": {
        "price_patterns": {
            "primary": r"\$(\d+(\.\d+)?) 12 month target ",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Appendix: Analyst Certification and Other Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Corbel(,Bold)?(,-Italic)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?(,Italic)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?(-Italic)?", "font_size": 10.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Cascend Securities -Historical-": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures: "],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 12.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Deutsche Bank": {
        "price_patterns": {
            "primary": r"Price Target \(USD\) (\d+(\.\d+)?)",
            "secondary": r"Price target (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
            "secondary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Appendix 1"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]UniversDeutscheBank-Regular", "font_size": 9.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "FinTrust Investment Advisors": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Fintrust Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures:"],
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Gilford Securities Inc": {
        "price_patterns": {
            "primary": r"\, \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rated: ([A-Za-z]+)",
            "secondary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": r"ANALYST CERTIFICATION",
        "font_patterns": [
            {"font_type":  r"Arial(MT)?(-BoldMT)?", "font_size": 10.02},
            {"font_type": "Not Available", "font_size": 10.00},
        ]
    },
    "Hilliard Lyons": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"-- NYSE\s+[–\-—]+\s+([A-Za-z]+)\s+[–\-—]+",
            "secondary":r"NYSE\s+[–\-—]+\s+([A-Za-z\- ]+?)(?=\s*[-–—]\d)"
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"Verdana(-Bold)?", "font_size": 9.00},
            {"font_type":r"TimesNewRomanPS(-BoldMT)?", "font_size": 10.98},
            {"font_type":r"Times(-Bold)?", "font_size": 10.98},
            ]
    },
    "IBI Investment House": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Recommendation: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "IFS Securities": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform|Outperform)\b",
        },
        "ending_patterns": [r"Important Investor Disclosures"],
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 11.04},
            {"font_type":r"TimesNewRomanPS(MT)?(-BoldMT)?", "font_size": 11.04},
        ]
    },
    "JP Morgan": {
        "price_patterns": {
            "primary": r"Price Target \([A-Za-z0-9\-]+\): \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?", "font_size": 9.60},
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?", "font_size": 10.08},
            {"font_type": r"[A-Z]+[+]TimesNewRomanPS(MT)?(,Bold)?", "font_size": 11.04},
            {"font_type": r"[A-Z]+[+]TimesNewRomanPS(MT)?(-BoldMT)?(-ItalicMT)?", "font_size": 10.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Mizuho Securities": {
        "price_patterns": {
            "primary": r"Price Target \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"IMPORTANT DISCLOSURES"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": r"Times-Roman(-Bold)?(-Italic)?(-BoldItalic)?", "font_size": 10.50},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Needham": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
            "secondary": r"PRICE TARGET: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Analyst Certification"
                            r"ANALYST CERTIFICATION"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Cambria(-Bold)?(-Regular)?", "font_size": 10.0},
            {"font_type": r"[A-Z]+[+]KeplerStd(-Bold)?", "font_size": 9.0},
        ]
    },
    "Phillip Securities": {
        "price_patterns": {
            "primary": r"TARGET PRICE USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Contact Information"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 10.0},
        ]
    },
    "Pivotal Research Group": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"RATING: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Appendix: Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Helvetica(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Arial", "font_size": 8.04},
        ]
    },
    "Spartan Capital Securities LLC": {
        "price_patterns": {
            "primary": r"T \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Wells Fargo": {
        "price_patterns": {
            "primary": r"\/Price Target: \$(\d+(\.\d+)?)",
            "secondary": r"Price Target\/Prior: \$(\d+(\.\d+)?)",
            "tertiary": r"\/\$(\d+(\.\d+)?)"
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)/\$",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Required Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]WellsFargoSans(-Light)?(,-SemiBold)?", "font_size": 9.00},
            {"font_type": r"[A-Z]+[+]Verdana(-Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01}
        ]
    }
}



In [196]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

def filter_valid_paragraphs(paragraphs):
    valid_paragraphs = []

    for paragraph in paragraphs:
        if check_validity(paragraph):  # Validate each paragraph
            valid_paragraphs.append(paragraph)
    return valid_paragraphs

def extract_text_with_format(page, provider):

    provider_patterns = patterns[provider]

    # Access patterns
    price_patterns = provider_patterns["price_patterns"]
    rating_patterns = provider_patterns["rating_patterns"]
    ending_patterns = provider_patterns["ending_patterns"]
    font_patterns = provider_patterns["font_patterns"]

    # Extract words with font and size details
    words = page.extract_words(extra_attrs=["fontname", "size"])

    # Round text sizes to 3 decimal places, moved it to lower part, hope this is correct
    """for word in words:
        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)"""

    # Sort words by vertical and horizontal position
    words.sort(key=lambda w: (w["top"], w["x0"]))

    paragraphs = []
    current_paragraph = []
    current_top = None

    rating = None
    price = None

    # Lookahead buffer for multi-word patterns
    lookahead_buffer = []

    for word in words:

        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)

        # Build lookahead buffer
        lookahead_buffer.append(word["text"])
        if len(lookahead_buffer) > 10:
            lookahead_buffer.pop(0)
        buffer_text = " ".join(lookahead_buffer)

        #print(buffer_text)

        # Extract rating
        if not rating:
            for pattern_key in ["primary", "secondary","tertiary"]:
                pattern = rating_patterns.get(pattern_key)
                if pattern:
                    rating_match = re.search(pattern, buffer_text)
                    if rating_match:
                        rating = rating_match.group(1)
                        break

        # Extract price
        if not price:
            for pattern_key in ["primary", "secondary"]:
                pattern = price_patterns.get(pattern_key)
                if pattern:
                    price_match = re.search(pattern, buffer_text)
                    if price_match:
                        price = price_match.group(1)
                        break

        # Check for ending pattern
        # Check if buffer matches any ending pattern
        for ending_pattern in ending_patterns:
            if re.search(ending_pattern, buffer_text):
                return filter_valid_paragraphs(paragraphs), True, rating, price

        # Match word against font patterns
        is_font_matched = False
        for font_pattern in font_patterns:
            font_type = font_pattern["font_type"]
            font_size = font_pattern["font_size"]
            if re.match(font_type, word["fontname"]) and word["size"] == font_size:
                is_font_matched = True
                #break

        if not is_font_matched:
            continue

        # Group words into paragraphs
        if current_top is None or abs(word["top"] - current_top) < 13:  # Adjust threshold as needed
            current_paragraph.append(word["text"])
        else:
            # New paragraph starts
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = [word["text"]]

        # Update the current top position
        current_top = word["top"]

    # Add the last paragraph
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    return filter_valid_paragraphs(paragraphs), False, rating, price


# NEW VERSION (16.11.2024)
def extract_metadata(filename, ticker_map):
    """
    Extract metadata (date, provider, ticker) from the filename using ticker_map.
    """
    # Extract the date (first 8 digits in the filename)
    date_match = re.match(r"(\d{8})", filename)
    if not date_match:
        return None, None, None, None, None
    date_str = date_match.group(1)
    date = datetime.strptime(date_str, "%Y%m%d")

    #print(f"fileName: {filename}")
    #print(f"date: {date}")

    # Look for the ticker in the filename
    for ticker in ticker_map.keys():
        ticker_pattern = f"_{ticker}_"  # Ensure ticker is surrounded by underscores
        if ticker_pattern in filename:
            # Extract the portion between date and ticker as the provider
            provider_section = filename.split(f"{date_str}_")[1].split(f"_{ticker}_")[0]
            provider = provider_section.replace('_', ' ')  # Replace underscores with spaces
            # Get company name and industry from the ticker_map
            company_name = ticker_map[ticker]['Company Name']
            industry = ticker_map[ticker]['Industry']
            return date, provider, ticker, company_name, industry

    # If no ticker is found, return None for ticker-related fields
    return date, None, None, None, None


In [197]:

# Define function to process all PDFs in the directory and store data in DataFrame
def extract_text_from_all_pdfs_to_dataframe(directory_path, provider_info, ticker_map):
    data = []
    id_counter = 1  # Initialize an ID counter
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):  # Process only PDF files
            file_path = os.path.join(directory_path, filename)

            
            # Extract metadata from filename
            date, provider, ticker, company_name, industry = extract_metadata(filename,ticker_map=ticker_map)
            all_paragraphs = []  
            first_rating = None
            first_price = None  
            stop_extraction = False

            with pdfplumber.open(file_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    if stop_extraction:
                        break  # Exit the loop if stop_extraction is set
            
                    # Extract data from the current page
                    paragraphs, stop_extraction, rating, price = extract_text_with_format(page, provider=provider)
                    
                    # Append paragraphs from the current page
                    all_paragraphs.extend(paragraphs)

                    # Capture the first non-None rating and price
                    if rating is not None and first_rating is None:
                        first_rating = rating
                    if price is not None and first_price is None:
                        first_price = price
                        
            # Add extracted data to the list
            data.append({
                "document_id": id_counter,  # Unique ID
                "filename": filename,
                "date": date,
                "provider": provider,
                "ticker": ticker,
                "company_name": company_name,
                "industry": industry,
                "paragraphs": all_paragraphs,
                "target_price":first_price,
                "rating": first_rating.lower() if first_rating else None
                })
            id_counter += 1  # Increment the ID counter for the next row

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

# Directory path
pdf_directory = "/Users/oskarroeske/Desktop/Analyst_Reports/Production"  # Replace with your actual folder path

# Run the function and store results in a DataFrame
df_reports = extract_text_from_all_pdfs_to_dataframe(pdf_directory, provider_info, ticker_map)

In [199]:
df_missing_paragraphs = df_reports[df_reports["paragraphs"].apply(lambda x: x == [])]

In [ ]:
import re

font_string = "Arial-BoldMT"
regex = r"Arial(MT)?(-BoldMT)?"

match = re.match(regex, font_string)
if match:
    print("Match found:", match.group())
else:
    print("No match found")

# Cleaning of paragraphs

In [ ]:
import re

def clean_paragraph(paragraph):
    # Remove email addresses
    paragraph = re.sub(r"\S+@\S+", "", paragraph)
    
    # Remove phone numbers
    paragraph = re.sub(r"\b\d{1,3}[-.\s]?\d{3}[-.\s]?\d{4}\b", "", paragraph)
    
    # Remove URLs
    paragraph = re.sub(r"http\S+|www\S+", "", paragraph)
    
    # Remove special characters (keep alphanumerics, spaces, and common punctuation)
    paragraph = re.sub(r"[^\w\s,.!?]", "", paragraph)
    
    # Remove multiple spaces or newlines
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    
    # Remove boilerplate phrases
    boilerplate_phrases = ["Disclaimer", "Confidential", "For internal use only"]
    for phrase in boilerplate_phrases:
        paragraph = paragraph.replace(phrase, "")
    
    # Remove short sentences (fewer than 10 words)
    if len(paragraph.split()) < 6:
        return None
    
    return paragraph

# Apply cleaning function
df_reports['paragraphs'] = df_reports['paragraphs'].apply(
    lambda paragraphs: [clean_paragraph(p) for p in paragraphs if clean_paragraph(p)]
)

# Drop rows where the `paragraphs` column is empty after cleaning
df_reports = df_reports[df_reports['paragraphs'].str.len() > 0]

df_reports["rating"] = df_reports["rating"].astype(str).apply(lambda x: x.lower() if x != "nan" else None)

# Get Date and Price for Dates

In [ ]:
df_reports.info()

In [ ]:
# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

In [ ]:
df_saved_reports = df_reports

In [ ]:
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return float(max_price), float(min_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data), float(next_available_data)
            else:
                return float('nan'), float('nan')  # Return NaN for missing data

    except KeyError:
        return float('nan'), float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'), float('nan')  # Return NaN for missing data

def calculate_prices(row):
    short_name = row['ticker']
    base_date = pd.to_datetime(row['date'], utc=True)

    # Calculate prices
    row['start price'] = get_stock_prices(short_name, base_date)[0]
    row['one day after'] = get_stock_prices(short_name, base_date + pd.DateOffset(days=1))[0]
    row['max price after 3 months'], row["min price after 3 months"] = get_stock_prices(
        short_name, base_date, base_date + pd.DateOffset(months=3)
    )
    row['max price after 6 months'], row['min price after 6 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=3), base_date + pd.DateOffset(months=6)
    )
    row['max price after 9 months'], row['min price after 9 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=6), base_date + pd.DateOffset(months=9)
    )
    row['max price after 12 months'], row['min price after 12 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=9), base_date + pd.DateOffset(months=12)
    )
    return row

# Apply the function to each row
df_saved_reports = df_saved_reports.apply(calculate_prices, axis=1)


In [ ]:
df_saved_reports.head(10)

In [ ]:
df_saved_reports["target_price"] = (
    df_saved_reports["target_price"]
    #Remove comma and transform to float for later operations
    .replace(",", "", regex=True) 
    .astype(float)                 
)

# Calculate performance of Target Prices

In [ ]:
#If the target_price is below the current price, we need to check if it was below the target_price
def classifcy_performance(df):
    if df["target_price"] > df["start price"]:
        df["tp reached after 3 months"] = df["target_price"] <= df["max price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] <= df["max price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] <= df["max price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["max price after 12 months"]
    else:
        df["tp reached after 3 months"] = df["target_price"] >= df["min price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] >= df["min price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] >= df["min price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["min price after 12 months"]

    return df

df_saved_reports = df_saved_reports.apply(classifcy_performance, axis=1)

# Clean/Filter Paragraphs

In [ ]:
df_saved_reports.value_counts("rating")

In [ ]:
#df_saved_reports.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_reports.csv")

In [ ]:
# Explode the cleaned list into separate rows
df_exploded = df_saved_reports.explode('paragraphs').reset_index(drop=True)

In [ ]:
df_exploded = df_exploded[["document_id","paragraphs"]]

In [ ]:
df_exploded['paragraph_id'] = df_exploded.groupby('document_id').cumcount() + 1

In [ ]:
df_exploded

In [ ]:
#df_exploded.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_paragraphs.csv")

# Turn paragraphs into sentences

In [ ]:
df_exploded

In [ ]:
import spacy
import pandas as pd

# Load spaCy language model
nlp = spacy.load("en_core_web_sm")

# Prepare an empty list to store the results
result = []

# Function to split text into sentences using spaCy
def split_into_sentences(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sents]

# Split each paragraph into sentences
for _, row in df_exploded.iterrows():
    doc_id = row['document_id']
    paragraph_id = row['paragraph_id']
    text = row['paragraphs']
    sentences = split_into_sentences(text)
    for sentence_id, sentence in enumerate(sentences, start=1):
        # Check if the sentence has more than 3 words
        if len(sentence.split()) >= 3:
            result.append({
                'document_id': doc_id,
                'paragraph_id': paragraph_id,
                'sentence_id': sentence_id,
                'sentence': sentence
            })

# Create a new DataFrame with the results
df_sentences = pd.DataFrame(result)

In [ ]:
df_sentences

In [ ]:
df_sentences.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_sentences.csv")

In [ ]:
#df_reports.to_csv("paragraphs.csv")